In [28]:
from sys import argv,stdin
import re

In [29]:
 """dot.notation access to dictionary attributes"""
class DotDict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

mydict = {'val':'it works'}
nested_dict = {'val':'nested works too'}
mydict = DotDict(mydict)
#print(mydict.val) # 'it works'

mydict.nested = DotDict(nested_dict)
#print(mydict.nested.val) # 'nested works too'


In [30]:
#for debug purposes
def actuallyPrintString(string):
    for line in string.split('\n'): print(line)

In [31]:
def removeComments(datastring):
    datastring = re.sub(re.compile("/\*.*?\*/",re.DOTALL ) , "", datastring)
    datastring = re.sub(re.compile("//.*?\n" ), "\n", datastring)
    return datastring

#strip leading and trailing whitespace of each line, and remove empty lines
def removeWhitespace(datastring):
    splitDelim = '\n'
    _datastring = [ line.strip() for line in datastring.split(splitDelim) ]
    _datastring = filter(lambda line: len(line) > 0, _datastring)
    datastring = splitDelim.join(_datastring)
    return datastring

#assumed that the string is already {}-balanced, it's your problem if you pass in a bad one
def matching_brace_index(string, start):
        i = start
        pseudostack = []
        found_complement = False
        while (not found_complement):
                if (string[i] == '{'):
                        pseudostack.append(string[i])
                if (string[i] == '}'):
                        pseudostack.pop()
                        if(len(pseudostack) == 0):
                                found_complement = True
                i += 1
        return i

def remove_section(string, start, end):
        upperbound = end
        lowerbound = start
        return string[0:lowerbound]+string[upperbound:len(string)]


def removeMethodBodies(data):
    firstbrace = data.index('{')
    lastbrace = matching_brace_index(data, firstbrace)
    inner = data[firstbrace+1:lastbrace-1]
    i = 0
    while i < len(inner):
        if inner[i]=='{':
            start = i; end = matching_brace_index(inner, i)
            inner = remove_section(inner, start, end)
        i += 1
    
    data = data[:firstbrace] + '{' + inner + '}' + data[lastbrace:]
    
    return data
    

In [95]:
#filename = '../src/EnergyCheckUtils.java'
filename = '../src/AsyncEnergyMonitorJavaSide.java'
#filename = '../src/JRAPL.java'
infile = open(filename,'r')
data = infile.read()
infile.close()
#actuallyPrintString(data)

In [96]:
data = removeComments(data)
data = removeMethodBodies(data)
data = removeWhitespace(data)
actuallyPrintString(data)

package jrapl;
import java.util.ArrayList;
import java.io.File;
import java.io.IOException;
import java.io.FileWriter;
public class AsyncEnergyMonitorJavaSide<E> extends JRAPL implements Runnable,AsyncMonitor
{
private final ArrayList<E> samples;
private int samplingRate;
private volatile boolean exit = false;
private Thread t = null;
private EnergyManager<E> energyManager;
public AsyncEnergyMonitorJavaSide(EnergyManager<E> eman, int s)
public void run()
public void start()
public void stop()
public void reInit()
public E[] getLastKSamples(int k)
public int getSamplingRate()
public void setSamplingRate(int s)
public int getNumReadings()
public void writeToFile(String fileName)
public String toString()
private String commaSeparated(double[] sample)
}


In [118]:
#don't break if you're removing indices that can't be accessed
def safePop(items, index):
    try:
        item = items.pop(index)
    except:
        item = None
    return item

def getClass(data):
    lines = data.split('\n')
    statement = [ d for d in lines if 'class' in d ][0]
    cls = DotDict()
    cls.name = statement.split('class ')[1].split()[0]
    if 'extends' in statement:
        cls.extends = statement.split(' extends ')[1].split()[0]
    if 'implements' in statement:
        cls.implements = statement.split(' implements ')[1].split()[0].split(',')
    return cls

def getPackage(data):
    lines = data.split('\n')
    return [ line for line in lines if line.startswith('package') ][0].split()[1].replace(';','')

def parseDeclaration(d): # same format for method and variable declarations (splitted by ' ')
    info = DotDict({})
    if d[0] in ['public','private','protected']:
        info.access = safePop(d,0)
    else: info.access = None
    info.name = safePop(d,-1)
    info.return_type = safePop(d,-1)
    info.other_info = ' '.join(d)
    return info

def getVariableList(data):
    firstbrace = data.index('{')
    lastbrace = matching_brace_index(data, firstbrace)
    inside_class = data[firstbrace+1:lastbrace-1].split('\n')
    variables = [ line for line in inside_class if not '(' in line and not ')' in line ]
    variables = filter(lambda x: len(x) > 0, variables)
    variables = [ re.sub('=.*$','',v) for v in variables ]
    variables = [ v.replace(';','').strip() for v in variables ]
    variables = [ v.split() for v in variables ]
    
    parsed_variables = [ parseDeclaration(v) for v in variables if len(v) > 1 ] # exclude one-word garbage that floated in
    
    return parsed_variables
    
def getMethodList(data):
    firstbrace = data.index('{')
    lastbrace = matching_brace_index(data, firstbrace)
    inside_class = data[firstbrace+1:lastbrace-1].split('\n')
    methods = [line for line in inside_class if '(' in line and ')' in line]
    methods = [re.sub('\(.*\)','()',method) for method in methods] #THIS IS JUST A HACK FOR NOW, DONT ACTUALLY DELETE
    methods = [ method.replace(';','') for method in methods ]
    methods = [ method.split() for method in methods ]
    
    parsed_methods = [ parseDeclaration(m) for m in methods if len(m) > 1 ] # exclude one-word garbage that floated in
    
    return parsed_methods

def getPublicVariables(data):
    return [ v for v in getVariableList(data) if v.access == 'public' ]

def getPrivateVariables(data):
    return [ v for v in getVariableList(data) if v.access == 'private' ]

def getPublicMethods(data):
    return [ m for m in getMethodList(data) if m.access == 'public' ]

def getPrivateMethods(data):
    return [ m for m in getMethodList(data) if m.access == 'private' ]

In [119]:
jclsInfo = DotDict()

jclsInfo.cls = getClass(data)
jclsInfo.package = getPackage(data)
jclsInfo.pub_vars = getPublicVariables(data)
jclsInfo.pri_vars = getPrivateVariables(data)
jclsInfo.pub_methods = getPublicMethods(data)
jclsInfo.pri_methods = getPrivateMethods(data)

#print(jclsInfo)
for k in jclsInfo.keys(): print(k,"\n",jclsInfo[k],"\n\n")

cls 
 {'name': 'AsyncEnergyMonitorJavaSide<E>', 'extends': 'JRAPL', 'implements': ['Runnable', 'AsyncMonitor']} 


package 
 jrapl 


pub_vars 
 [] 


pri_vars 
 [{'access': 'private', 'name': 'samples', 'return_type': 'ArrayList<E>', 'other_info': 'final'}, {'access': 'private', 'name': 'samplingRate', 'return_type': 'int', 'other_info': ''}, {'access': 'private', 'name': 'exit', 'return_type': 'boolean', 'other_info': 'volatile'}, {'access': 'private', 'name': 't', 'return_type': 'Thread', 'other_info': ''}, {'access': 'private', 'name': 'energyManager', 'return_type': 'EnergyManager<E>', 'other_info': ''}] 


pub_methods 
 [{'access': 'public', 'name': 'AsyncEnergyMonitorJavaSide()', 'return_type': None, 'other_info': ''}, {'access': 'public', 'name': 'run()', 'return_type': 'void', 'other_info': ''}, {'access': 'public', 'name': 'start()', 'return_type': 'void', 'other_info': ''}, {'access': 'public', 'name': 'stop()', 'return_type': 'void', 'other_info': ''}, {'access': 'public', 